In [1]:
import os
os.chdir(r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data")
import numpy as np
import pandas as pd
#!pip install openpyxl

In [2]:
#product matrix
matrix = pd.read_csv (r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\SO_data (4).csv')
matrix = matrix[['plu_code', 'in_matrix', 'address_text']]
matrix = matrix.drop_duplicates()


C:\Users\maiguann\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#directory (products' prices)
current_prices = pd.read_excel(r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\справочник\current_prices.xlsx', sheet_name = 'E2E X5')
current_prices = current_prices.iloc[3: , :]
current_prices=current_prices.rename(columns=current_prices.iloc[0]).drop(current_prices.index[0])
current_prices['price'] = current_prices['Outlet Stock, RUR'].divide(current_prices['Outlet Stock, Qty']+.000000000000000000000000001)
current_prices['price']=current_prices['price'].replace([np.inf,-np.inf], np.nan)
current_prices['price']=current_prices['price'].replace(0,np.nan)
current_prices['Product Code']=current_prices['Product Code'].astype(int)
old_prices = pd.read_excel(r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\справочник\ZREP-PLU-price.xlsx', sheet_name = 'ZREP-PLU-Price')
current_prices = pd.merge(current_prices, old_prices, left_on = 'Product Code', right_on = 'PLU', how='outer')
current_prices['price'].fillna(current_prices['BASE PRICE, PC'], inplace=True)
current_prices['Product Code'].fillna(current_prices['PLU'], inplace=True)
current_prices = current_prices[['Product Code','price']]
current_prices=current_prices.drop_duplicates(subset =['Product Code'] )
current_prices = current_prices.dropna(subset = ['price'])

In [27]:
#data
stock = pd.read_csv (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\stock MW 2609.csv")
stock.drop('Unnamed: 0', inplace=True, axis=1)
sales = pd.read_csv (r"C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\data_csv\sales MW 2609.csv")
sales.drop('Unnamed: 0', inplace=True, axis=1)
#data columns
data_sales = [col for col in sales if col.startswith('2021')]
data_stock = [col for col in stock if col.startswith('2021')]
stock=pd.merge(stock, matrix,on = ('plu_code', 'address_text'), how = 'left')
stock = stock[stock['in_matrix']==1]
sales=pd.merge(sales, matrix,on = ('plu_code', 'address_text'), how = 'left')
sales = sales[sales['in_matrix']==1]


C:\Users\maiguann\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
stock.drop('in_matrix', inplace=True, axis=1)
sales.drop('in_matrix', inplace=True, axis=1)

## Stock

### filtering out blocked plu

In [29]:
stock = stock[stock['plu_name']!="M&Ms Драж.MAXI с молочным шоколадом 70г"]
stock = stock[stock['plu_name']!="M&Ms Драже MAXI с арахисом 70г"]
stock = stock[stock['plu_name']!="M&MS Драже КРИСПИ хр.шар.мол.шок.гл.110г"]
stock = stock[stock['plu_name']!="M&Ms Драже с арахисом 130г"] 
stock = stock[stock['plu_name']!="M&Ms Драже шоколадный 130г"] 
stock = stock[stock['plu_name']!="SNICKERS SUP.Батончик шок.ж.ар/кар/н.95г"]

In [30]:
stock['oos_days']=(stock[data_stock]==0).sum(axis=1)

In [31]:
stock_list = stock[stock['oos_days']>0]
stock_list.to_excel(r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\OOS\list_oos_2609.xlsx')

In [32]:
#number of outlets
stock_total = stock.groupby(["macro_name", "plu_code", "plu_name", "assort_level"]).agg({'werks':'size','oos_days':'sum'}).reset_index()

In [33]:
stock_grouped = stock.groupby(["macro_name", "plu_code", "plu_name", "assort_level"])['oos_days'].value_counts().unstack(fill_value=0).reset_index()#.agg({'werks':'size','oos_days':'sum'})

In [34]:
stock_grouped = pd.merge(stock_grouped, stock_total)

## Sales

In [35]:
stock_melt = stock.melt(id_vars = ('werks', 'address_text', 'macro_name', 'assort_level', 'status', 'brand_name', 'plu_name', 'plu_code', 'oos_days'), var_name = 'date', value_name = 'stock')

In [36]:
sales = sales.melt(id_vars = ('werks', 'address_text', 'macro_name', 'assort_level', 'status', 'brand_name', 'plu_name', 'plu_code'), var_name = 'date', value_name = 'sales')
sales = pd.merge(sales, stock_melt, how = 'left')
sales = sales[sales['stock'] != 0]

In [37]:
#price
sales = pd.merge(sales, current_prices, left_on = 'plu_code', right_on = 'Product Code', how = 'left')

In [39]:
sales['assort_level'] = sales['assort_level'].astype('object')
sales['plu_code'] = sales['plu_code'].astype('object')
sales['sales'] = sales['sales'].astype('float')

In [41]:
#stock
sales_grouped = sales.groupby(["macro_name", "plu_code", "plu_name", "assort_level", 'brand_name', 'werks']).agg({'sales':'sum', 'oos_days':'mean', 'price' :'mean'}).reset_index()
sales_grouped['daily_sales'] = (sales_grouped['sales']*sales_grouped['price'])/(7-sales_grouped['oos_days'])
sales_grouped = sales_grouped.groupby(["macro_name", "plu_code", "plu_name", "assort_level", 'brand_name']).agg({'werks':'size','sales':'sum', 'daily_sales':'mean','price' :'mean'}).reset_index()


In [42]:
sales_grouped = sales_grouped[['macro_name', 'plu_code', 'plu_name', 'assort_level', 'brand_name','sales', 'daily_sales', 'price']]


## Merging oos dataset and daily sales dataset

In [43]:
final = pd.merge(stock_grouped, sales_grouped, on = ('macro_name', 'plu_code', 'plu_name', 'assort_level'), how = 'left')

In [44]:
final['lost_sales'] = (final[1]*final['daily_sales']) + (final[2]*final['daily_sales'] *2)+(final[3]*final['daily_sales'] *3)+(final[4]*final['daily_sales'] *4)+(final[5]*final['daily_sales'] *5)+(final[6]*final['daily_sales'] *6)+(final[7]*final['daily_sales'] *7)


In [45]:
final.to_excel(r'C:\Users\maiguann\OneDrive - Mars Inc\Documents\MW alert system DC\OOS\oos_2609.xlsx')